In [362]:
import pandas as pd
import numpy as np
from datetime import timedelta

import matplotlib.pyplot as plt

import evaluation
import make_prediction as mp

from scipy import stats
import math

In [729]:
df= pd.read_csv('../data/training.csv', parse_dates= True)

In [822]:
sample=df.sample(frac=0.001)

In [975]:
test= pd.read_csv('../data/testing.csv', parse_dates=True)

In [800]:
test.groupby('product_id').size()

product_id
s50045366     448
s50096611     628
s50131077     438
s50262812     975
s50324991     761
             ... 
x8385515     2876
x8396038      638
x8402864      486
x899896       896
x9683723     1664
Length: 600, dtype: int64

## sample 

In [842]:
# creating difference features 

#sample['pct_change']=sample['sensor_t4'].diff()

def pct_change(variable,df):
    for var in variable:
        if var in df.columns:
            df[var+'_pct_change']=df[var].pct_change()
        else:
            print('Column does not exist')

In [843]:
slist =['sensor_t4','sensor_t1','sensor_t5','sensor_q1','sensor_s4','sensor_t6','sensor_q2',
       'sensor_t9','sensor_s3','sensor_t2','sensor_t7','sensor_s1','sensor_t3','sensor_s2','sensor_t8']

s=pct_change(slist,df)

In [976]:
st=pct_change(slist,test)

### Some products are missing process 
#### Like s50016128 , this product has only gone throught process B and C 

### Aggregration

In [436]:
def std(x):
    f=np.std(x)
    return f

def mean(x):
    f=np.mean(x)
    return f

def entropy(s):
    px = s.value_counts() / s.shape[0]
    lpx = np.log2(px)
    ent = -1.0*(px*lpx).sum()
    return ent 

def var(x):
    return np.var(x)

##### Numerical data from sensors

In [832]:
aggregrations ={
                'sensor_t4':[min,max,mean,var,entropy],
                'sensor_t1':[min,max,mean,var,entropy],
                'sensor_t5':[min,max,mean,var,entropy],
                'sensor_q1':[min,max,mean,var,entropy],
                'sensor_s4':[min,max,mean,var,entropy],
                'sensor_t6':[min,max,mean,var,entropy],
                'sensor_s3':[min,max,mean,var,entropy],
                'sensor_q2':[min,max,mean,var,entropy],
                'sensor_t9':[min,max,mean,var,entropy],
                'sensor_t2':[min,max,mean,var,entropy],
                'sensor_s3':[min,max,mean,var,entropy],
                'sensor_t7':[min,max,mean,var,entropy],
                'sensor_s2':[min,max,mean,var,entropy],
                'sensor_t1':[min,max,mean,var,entropy],
                'sensor_t3':[min,max,mean,var,entropy],
                'sensor_s2':[min,max,mean,var,entropy],
                'sensor_t8':[min,max,mean,var,entropy],
                'timestamp':[min,max],
                'flow_id':['last'],
                'lot_id':['last'],
                'flag_b2':[entropy],
                'flag_c2':[entropy],
                'flag_e': [entropy],
                'flag_a2':[entropy],
                'flag_a4':[entropy],
                'flag_b1':[entropy],
                'flag_d': [entropy],
                'flag_a1':[entropy],
                'flag_b3':[entropy],
                'flag_b4':[entropy],
                'flag_c1':[entropy],
                'flag_a3':[entropy],
                'flag_a5':[entropy],
                'flag_b2':['last','first',mean],
                'flag_c2':['last','first',mean],
                'flag_e':['last','first',mean],
                'flag_a2':['last','first',mean],
                'flag_a4':['last','first',mean],
                'flag_b1':['last','first',mean],
                'flag_d':['last','first',mean],
                'flag_a1':['last','first',mean],
                'flag_b3':['last','first',mean],
                'flag_b4':['last','first',mean],
                'flag_c1':['last','first',mean],
                'flag_a3':['last','first',mean],
                'flag_a5':['last','first',mean],
                'sensor_t4_pct_change':[min,max],
                'sensor_t1_pct_change':[min,max],
                'sensor_t5_pct_change':[min,max],
                'sensor_q1_pct_change':[min,max],
                'sensor_s4_pct_change':[min,max],
                'sensor_t6_pct_change':[min,max],
                'sensor_s3_pct_change':[min,max],
                'sensor_q2_pct_change':[min,max],
                'sensor_t9_pct_change':[min,max],
                'sensor_t2_pct_change':[min,max],
                'sensor_s3_pct_change':[min,max],
                'sensor_t7_pct_change':[min,max],
                'sensor_s2_pct_change':[min,max],
                'sensor_t1_pct_change':[min,max],
                'sensor_t3_pct_change':[min,max],
                'sensor_s2_pct_change':[min,max],
                'sensor_t8_pct_change':[min,max]
                
  }


In [829]:
# apply aggregation functions
def aggregation(df):
    group_obj=df.groupby(['product_id','process'])
    '''change the groupby object to full df'''
    f=group_obj.agg(aggregrations)

# renaming columns with multiIndex levels 
    f.columns = ["_".join(x) for x in f.columns.ravel()]
#unstacking the agg features to create separate feature for each process
    funstack=f.unstack(level=1)
# Renaming these features 
    funstack.columns = ["_".join(x) for x in funstack.columns.ravel()]
#flattening the index 
    funstack.reset_index(inplace=True)
    return funstack

In [844]:
flat=aggregation(df)

In [977]:
flat_test=aggregation(test)

In [978]:
flat_test.shape

(600, 565)

#### adding time duration 
##### count total seconds spent on each process 
######tot_seconds_proccessA = 290
######tot_seconds_proccessB = 456
sample['timestamp_seconds']=sample.loc['timestamp'].dt.second.cumsum()

In [487]:
# converting string to datetime to calculate time spent
timestamplist =['timestamp_max_A','timestamp_min_A','timestamp_max_B','timestamp_min_B','timestamp_max_C',
                 'timestamp_min_C','timestamp_max_D','timestamp_min_D']

# Fill the Missing timestamps as 0 
## Assumption: If no time was spent on the stage then it is meaningful 0


In [979]:
#funstack.loc[:,timestamplist].apply(pd.to_datetime)
for timecolumn in timestamplist:
    flat_test[timecolumn]= flat_test[timecolumn].apply(lambda x : pd.to_datetime(x))

In [847]:
# Fill the Missing timestamps as 0 
## Assumption: If no time was spent on the stage then it is meaningful 0
flat['timeSpent_A'] = (flat['timestamp_max_A'] - flat['timestamp_min_A']).fillna(pd.Timedelta(seconds=0))/np.timedelta64(1, 's')
flat['timeSpent_B'] = (flat['timestamp_max_B'] - flat['timestamp_min_B']).fillna(pd.Timedelta(seconds=0))/np.timedelta64(1, 's')
flat['timeSpent_C'] = (flat['timestamp_max_C'] - flat['timestamp_min_C']).fillna(pd.Timedelta(seconds=0))/np.timedelta64(1, 's')
flat['timeSpent_D'] = (flat['timestamp_max_D'] - flat['timestamp_min_D']).fillna(pd.Timedelta(seconds=0))/np.timedelta64(1, 's')

In [980]:
# Fill the Missing timestamps as 0 
## Assumption: If no time was spent on the stage then it is meaningful 0
flat_test['timeSpent_A'] = (flat_test['timestamp_max_A'] - flat_test['timestamp_min_A']).fillna(pd.Timedelta(seconds=0))/np.timedelta64(1, 's')
flat_test['timeSpent_B'] = (flat_test['timestamp_max_B'] - flat_test['timestamp_min_B']).fillna(pd.Timedelta(seconds=0))/np.timedelta64(1, 's')
flat_test['timeSpent_C'] = (flat_test['timestamp_max_C'] - flat_test['timestamp_min_C']).fillna(pd.Timedelta(seconds=0))/np.timedelta64(1, 's')
flat_test['timeSpent_D'] = (flat_test['timestamp_max_D'] - flat_test['timestamp_min_D']).fillna(pd.Timedelta(seconds=0))/np.timedelta64(1, 's')

In [848]:
flat.fillna(0,inplace=True)

In [981]:
flat_test.fillna(0,inplace=True)

In [982]:
flat_test.shape

(600, 569)

In [983]:
flat_test['Total_time_spent'] = flat_test.loc[:,
                                    ['timeSpent_A','timeSpent_B','timeSpent_C','timeSpent_D']].sum(axis=1)

In [724]:
### Extract time of the process 
### Hpothesis : may be parts going through production during day have high loq qc score due to heat generated by friction

### Hypothesis 2: total time spend by a product on a particular process will tell us how much deviation is there in the current product 
'''
flat['max_timespent_A']= np.max(flat['timeSpent_A'])
flat['max_timespent_B']= np.max(flat['timeSpent_B'])
flat['max_timespent_C']= np.max(flat['timeSpent_C'])
flat['max_timespent_D']= np.max(flat['timeSpent_D'])

flat'''

"\nflat['max_timespent_A']= np.max(flat['timeSpent_A'])\nflat['max_timespent_B']= np.max(flat['timeSpent_B'])\nflat['max_timespent_C']= np.max(flat['timeSpent_C'])\nflat['max_timespent_D']= np.max(flat['timeSpent_D'])\n\nflat"

## Joining the Target label

In [181]:
labels = pd.read_csv('../data/training_label.csv')

In [851]:
merged= flat.merge(labels, left_on='product_id', right_on='product_id', how='left')

In [206]:
merged.to_csv('../data/trainig_dataV11.csv', index=False)

In [703]:
holdout= flat.merge(labels, left_on='product_id', right_on='product_id', how='left')

## Train test split 

In [852]:
merged.shape

(4405, 571)

In [861]:
holdout = merged.sample(frac=0.2, random_state=42)

In [690]:
train=merged.drop(holdout.index)

In [863]:
print("training shape",train.shape)
print("external set shape",holdout.shape)

training shape (3524, 523)
external set shape (881, 572)


In [862]:
holdout['row_id']= holdout.reset_index().index

In [579]:
holdout_products = holdout['product_id']

In [864]:
holdout['product_id']

2638    x18338263
668     s64647679
1213    s76786960
478     s59877646
530     s61083173
          ...    
4386     x9631590
321     s56897875
2309    x11390046
888     s69676814
655     s64322497
Name: product_id, Length: 881, dtype: object

In [865]:
holdout_products

2638    x18338263
668     s64647679
1213    s76786960
478     s59877646
530     s61083173
          ...    
4386     x9631590
321     s56897875
2309    x11390046
888     s69676814
655     s64322497
Name: product_id, Length: 881, dtype: object

## test data

In [984]:
test_data= flat_test

In [985]:
test_data.columns

Index(['product_id', 'sensor_t4_min_A', 'sensor_t4_min_B', 'sensor_t4_min_C',
       'sensor_t4_min_D', 'sensor_t4_max_A', 'sensor_t4_max_B',
       'sensor_t4_max_C', 'sensor_t4_max_D', 'sensor_t4_mean_A',
       ...
       'sensor_t8_pct_change_min_D', 'sensor_t8_pct_change_max_A',
       'sensor_t8_pct_change_max_B', 'sensor_t8_pct_change_max_C',
       'sensor_t8_pct_change_max_D', 'timeSpent_A', 'timeSpent_B',
       'timeSpent_C', 'timeSpent_D', 'Total_time_spent'],
      dtype='object', length=570)

## DataRobot - Project building

In [41]:
import datarobot as dr

In [853]:
dr.Client(token='NWVlYWVmNzI4YjcxY2IyODE2ZTJhOTg5OnhSdjlqSEZ2NlU5cHc4WWQwZlh1RHNkbGZGREE0dlQzQnB5L3k2V2d3a2c9', endpoint='https://app.datarobot.com/api/v2/')

In [697]:
partition=dr.RandomCV(reps=4, holdout_pct=0, seed=42)

In [854]:
project = dr.Project.create(sourcedata=merged,
                           project_name='LUSY_Training_only_V5_FULL')
print('Project ID: {}'.format(project.id))

Project ID: 5f2b9c98ce243f0a4ec5db90


## prepare the external holdout set 

run the preprocessing steps

In [988]:
#load the project
proj=dr.Project.get('5f29d92cd6e13f37136dcc0a')
# chose the model 
model = dr.Model.get(project=proj.id,
                     model_id='5f2cb885735d1f06c8953475')
#upload the external dataset 
external= proj.upload_dataset(test_data)

In [987]:
proj.get_datasets()[1]

[PredictionDataset('predict.csv'), PredictionDataset('predict.csv')]

In [989]:
proj_id=proj.id
model_id=model.id
ext_id= external.id
#ext_id= proj.get_datasets()[0].id

In [629]:
def get_predictions_(project_id,model_id,ext_id):
    #upload external set
    ## Check if there is an external set already. If yes, then get the dataset.id instead of uploading a new one 
    model = dr.Model.get(project=proj_id,
                     model_id=model_id)
    predict_job=model.request_predictions(ext_id)
    predict_job.wait_for_completion()
    #get predictions
    predictions = predict_job.get_result_when_complete()
    #predictions['exp_predictions'] = np.exp(predictions['prediction'])
    #predictions['row_id']= predictions.reset_index().index
    return predictions

In [619]:
def join_results(predictions,actual_df):
    result=actual_df.merge(predictions,left_on='row_id',right_on='row_id', how='left')
    #holdout_with_target= result.merge(labels, left_on='product_id', right_on='product_id', how='left')
    return amape(result['qc_reading'],result['prediction'])

In [990]:
r =get_predictions_(proj_id,model_id,ext_id)

In [991]:
r.shape

(600, 2)

In [992]:
score = join_results(r,holdout)
score

<ipython-input-92-8330c5c7e946>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual[a] = 2000000


0.4729

In [ ]:
0.0825

In [ ]:
## current best scor e
0.1287
bestmodelid='5f28d4b2ce243f3780c8546c'
bestprojID='5f28a52d6c6a7a330eaff775'

In [901]:
best_models ={'projid':'5f29d92cd6e13f37136dcc0a','model_id':'5f2b898fce243f09f3c5dadf','score':0.0825}

In [902]:
best_models.update('')

{'projid': '5f29d92cd6e13f37136dcc0a',
 'model_id': '5f2b898fce243f09f3c5dadf',
 'score': 0.0825}

## Evaluation 


#### Beat 0.121 score 

In [92]:
def amape(actual,pred):
    #actual, pred = np.array(actual), np.array(pred)
    for a in actual:
        if a < 2000000:
            actual[a] = 2000000
            #mape =np.round(np.mean(np.abs((actual - pred) / actual)),4)
        else:
            None
            mape =np.round(np.mean(np.abs((actual - pred) / actual)),4)
            return mape


In [965]:

scorecard = {'ProjectID': ['5f281d9bce243f25ac54c7a9'],
        'model_id': ['5f281f78984a434cced832fa'],
             'score':[0.181],
     'insample':['yes']
        }

scores = pd.DataFrame(scorecard, columns = ['ProjectID', 'model_id','score','insample'])

print (scores)

                  ProjectID                  model_id  score insample
0  5f281d9bce243f25ac54c7a9  5f281f78984a434cced832fa  0.181      yes


In [970]:
#scores.append({'ProjectID' : proj.id , 'model_id' : model.id,'score':score,'insample':'no' } )

TypeError: Can only append a dict if ignore_index=True

### submitted models 

V4 FULL projec - https://app.datarobot.com/projects/5f29d92cd6e13f37136dcc0a/models/5f2a2104ce243f4ab7c854a6/blueprint


In [958]:
scores

,ProjectID,model_id,score,insample,2
0,5f2902676c6a7a2df18d88c3,5f2caee19071ae0609b632cc,0.181,yes,0
